Khai báo các thư viện cần thiết

In [82]:
import os
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

Khai báo đường dẫn tệp dữ liệu

In [83]:
data_path = 'D:/Code/python/mat3533/project/data/'

train_images_path = os.path.join(data_path, 'train-images-idx3-ubyte.gz')
train_labels_path = os.path.join(data_path, 'train-labels-idx1-ubyte.gz')

test_images_path = os.path.join(data_path, 't10k-images-idx3-ubyte.gz')
test_labels_path = os.path.join(data_path, 't10k-labels-idx1-ubyte.gz')

Hàm lấy dữ liệu

In [84]:
def get_mnist_data(images_path, labels_path, num_images, shuffle=False, _is=True, image_size=28):
    """
    This shuffle param is active when .gz is downloaded at:
    - 'http://yann.lecun.com/exdb/mnist/'
    - This function return random num_images in 60000 or 10000
    """
    # read data
    import gzip # to decompress gz (zip) file
    # open file training to read training data
    f_images = gzip.open(images_path,'r')
    # skip 16 first bytes because these are not data, only header infor
    f_images.read(16)
    # general: read num_images data samples if this parameter is set;
    # if not, read all (60000 training or 10000 test)
    real_num = num_images if not shuffle else (60000 if _is else 10000)
    # read all data to buf_images (28x28xreal_num)
    buf_images = f_images.read(image_size * image_size * real_num)
    # images
    images = np.frombuffer(buf_images, dtype=np.uint8).astype(np.float32)
    images = images.reshape(real_num, image_size, image_size,)
    # Read labels
    f_labels = gzip.open(labels_path,'r')
    f_labels.read(8)
    labels = np.zeros((real_num)).astype(np.int64)
    # rearrange to correspond the images and labels
    for i in range(0, real_num):
        buf_labels = f_labels.read(1)
        labels[i] = np.frombuffer(buf_labels, dtype=np.uint8).astype(np.int64)

    # shuffle to get random images data
    if shuffle is True:
        rand_id = np.random.randint(real_num, size=num_images)
        images = images[rand_id, :]
        labels = labels[rand_id,]
    # change images data to type of vector 28x28 dimentional
    images = images.reshape(num_images, image_size * image_size)
    return images, labels

Lấy dữ liệu train, test từ tệp dữ liệu

In [85]:
train_images, train_labels = get_mnist_data(
    train_images_path, train_labels_path, 60000)
test_images, test_labels = get_mnist_data(
    test_images_path, test_labels_path, 10000, _is=False)
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [71]:
all_images = np.concatenate((train_images, test_images), axis=0)

In [86]:
pca = PCA(n_components=100)
pca.fit(all_images)
X_train = pca.transform(train_images)
X_test = pca.transform(test_images)
print(X_train.shape)
print(X_test.shape)

(60000, 100)
(10000, 100)


In [73]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

In [87]:
model = Sequential()

model.add(Flatten(input_shape = (100,)))
model.add(Dense(128,activation = 'relu'))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(32,activation = 'relu'))
model.add(Dense(10,activation = 'softmax')) 

c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [88]:
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_10 (Flatten)            │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 128)            │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,594 (92.16 KB)

 Trainable params: 23,594 (92.16 KB)

 Non-trainable params: 0 (0.00 B)

In [89]:
model.compile(loss = 'sparse_categorical_crossentropy', 
              optimizer = 'Adam', 
              metrics = ['accuracy'])

In [90]:
model.fit(X_train,train_labels,epochs= 10 , validation_split = .2)

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.6938 - loss: 11.0520 - val_accuracy: 0.8791 - val_loss: 0.5382
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8888 - loss: 0.4550 - val_accuracy: 0.9125 - val_loss: 0.3643
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9260 - loss: 0.2702 - val_accuracy: 0.9319 - val_loss: 0.2839
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9422 - loss: 0.2137 - val_accuracy: 0.9382 - val_loss: 0.2471
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9555 - loss: 0.1612 - val_accuracy: 0.9469 - val_loss: 0.2110
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9644 - loss: 0.1237 - val_accuracy: 0.9549 - val_loss: 0.1713
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9677 - loss: 0.1070 - val_accuracy: 0.9495 - val_loss: 0.1889
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9709 - loss: 0.0982 -

In [91]:
y_pred = model.predict(X_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step


In [93]:
y_pred = y_pred.argmax(axis = 1)

In [94]:
y_pred

array([7, 2, 1, ..., 4, 5, 6], dtype=int64)

In [95]:
accuracy = accuracy_score(test_labels, y_pred)
print("Accuracy:", accuracy)

confusion_matrix = confusion_matrix(test_labels, y_pred)
print("Confusion Matrix:")
print(confusion_matrix)

precision = precision_score(test_labels, y_pred, average='macro')
print("Precision:", precision)

recall = recall_score(test_labels, y_pred, average='macro')
print("Recall:", recall)

Accuracy: 0.9621
Confusion Matrix:
[[ 957    0    4    1    3    3    6    5    1    0]
 [   0 1110    3    2    9    1    5    1    4    0]
 [   1    0  984    8    9    2    3    7   17    1]
 [   0    0    6  979    0    7    0    3   12    3]
 [   1    1    2    0  943    0    9    0    3   23]
 [   2    0    0   20    0  847   12    1    6    4]
 [   8    2    0    0    5    5  937    0    1    0]
 [   2    2   12    5    9    0    0  971    4   23]
 [   2    1    5    1    5   15    5    4  928    8]
 [   2    1    1    9   15    6    0    4    6  965]]
Precision: 0.9617415083829917
Recall: 0.9618932588038834
